In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from os import listdir

from torchvision import transforms, utils

from torchvision import transforms 

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

import glob

In [4]:
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [5]:
def stripping(liste):
    return [i.strip() for i in liste]

In [8]:
df=pd.read_csv('data/over.csv',converters={'list_alle': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'list_normen': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'list_gesamt': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
})

df['list_alle']=df['list_alle'].progress_apply(stripping)
df['list_normen']=df['list_normen'].progress_apply(stripping)
df['list_gesamt']=df['list_gesamt'].progress_apply(stripping)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df=df.reset_index(drop=True)

  0%|          | 0/19500 [00:00<?, ?it/s]

  0%|          | 0/19500 [00:00<?, ?it/s]

  0%|          | 0/19500 [00:00<?, ?it/s]

In [9]:
df.head()

,text,topic,gender,age,topic_num,percent,list_alle,list_normen,list_gesamt
0,This blog is being posted due to the fact that...,Student,male,14,7,0.4414019715224534,"[0.0, 0.0, 0.16700000000000004, 0.0, 0.021, 0....","[0.0, 0.0, 0.0, 0.0, 0.055999999999999994, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."
1,So I have a big fucking interview tomorrow for...,Student,male,15,20,0.3124452234881683,"[0.176, 0.0, 0.083, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, 0.849, 0.0, 0.0, 0.023, 0.0, 0.153000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."
2,I was reminded just now of the time Ashley and...,Student,female,17,27,0.20397382322794297,"[1.863999999999999, 0.05, 0.5950000000000003, ...","[0.0, 5.121999999999999, 0.0, 0.0, 0.417000000...","[0.004, 0.0, 0.0, 0.018000000000000002, 0.0, 0..."
3,I was checking up on my cousin Dylan and Fanni...,Student,female,23,20,0.2786579683131407,"[0.0, 0.0, 0.08300000000000002, 0.08, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.069, 0.0, 0.042, 0.0, 0...","[0.0, 0.0, 0.003, 0.006, 0.0, 0.0, 0.0, 0.0, 0..."
4,for the NME interview click urlLink part 1 and...,Student,female,23,31,0.9468664850136239,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0..."


In [96]:
over=pd.DataFrame(data=df['list_alle'].tolist(),columns=[f'Topic_{x}' for x in range(len(df['list_alle'][0]))])

In [97]:
over.head()

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,...,Topic_28,Topic_29,Topic_30,Topic_31,Topic_32,Topic_33,Topic_34,Topic_35,Topic_36,Topic_37
0,0.0,0.0,0.16700000000000004,0.0,0.021,0.0,0.0,0.403,0.0,0.022,...,0.0,0.15600000000000003,0.0,0.0,0.022,0.0,0.0,0.0,0.0,0.0
1,0.176,0.0,0.083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016,0.087,0.008,0.112,0.0,0.0
2,1.863999999999999,0.05,0.5950000000000003,0.0,0.0,0.189,0.216,0.0,0.0,0.0,...,0.102,0.0,0.07,0.0,0.0,0.033,0.0,0.14800000000000002,0.064,0.136
3,0.0,0.0,0.08300000000000002,0.08,0.0,0.056,0.0,0.41700000000000004,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112,0.017,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.39,0.0,0.0,0.0,0.0,0.0,0.04


In [103]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=42

In [104]:
over = over.astype(float)

In [109]:
df['topic_num'] = pd.factorize(df['topic'])[0]

In [131]:
X_NS=over
y_NS=df['topic_num']
X_train_NS, X_test_NS, y_train_NS, y_test_NS = train_test_split( X_NS, y_NS, test_size=0.20, random_state=RANDOM_SEED)

In [149]:
y_test_NS

9960     19
13888    27
8114     16
3508      7
1027      2
         ..
7484     14
16246    32
17534    35
10285    20
3503      7
Name: topic_num, Length: 3900, dtype: int64

In [133]:
from sklearn.preprocessing import StandardScaler, RobustScaler
# for cols in over.columns:
#     over[cols] = StandardScaler()

In [134]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ct = ColumnTransformer([
        ('somename', StandardScaler(), [x for x in over.columns])
    ], remainder='passthrough')

X_train=ct.fit_transform(X_train_NS)

X_test=ct.fit_transform(X_test_NS)

In [118]:
#X_train=torch.from_numpy(X_train).float()
#X_test=torch.from_numpy(X_test).float()

In [127]:
#y_train=torch.FloatTensor(y_train_NS)
#y_test=torch.FloatTensor(y_train_NS)

--> neuronales netzt --> den tensor input --> mit convid 2--> basiernd davon ergebnisse bekommen 

In [122]:
y_train

tensor([36., 16.,  0.,  ..., 10.,  1., 31.])

In [135]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# create a tensor input list


In [136]:
model = Net(input_size=38, hidden_size=128, num_classes=38)
print(model)

Net(
  (fc1): Linear(in_features=38, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=38, bias=True)
)


In [137]:
learning_rate = 0.01
momentum = 0.9
n_epochs = 10

optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)

loss_func = torch.nn.MSELoss()

In [165]:
def train(columns_df,scaled_features,epoch):
    batch_idx = 0
    for row in range(len(torch.from_numpy(columns_df).float())):
        features = torch.from_numpy(columns_df).float()[row]
        price = torch.FloatTensor(scaled_features)[row]
        #print(price)

        if torch.cuda.is_available():
            image = image.cuda()
            features = features.cuda()
            
        output = model(features)
        #output = output.reshape(38)
        loss = loss_func(output, price)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
    

In [166]:
for epoch in range(n_epochs):
    # train 
    print('Start Epoch: ',epoch)
    train(X_train,y_train_NS,epoch)

Start Epoch:  0
Start Epoch:  1
Start Epoch:  2
Start Epoch:  3
Start Epoch:  4
Start Epoch:  5
Start Epoch:  6
Start Epoch:  7
Start Epoch:  8
Start Epoch:  9


In [150]:
torch.FloatTensor(y_test_NS.values)


tensor([19., 27., 16.,  ..., 35., 20.,  7.])

In [170]:
import math
def reg_report(true, pred, name='Test'):
    print("\n{} Results :\n".format(name))
    print("RSS :",sum((pred-true)**2))
    print("RSE :",math.sqrt(sum((pred-true)**2)*(1/(len(pred)-2))))
    print("TSS :",sum((true-true.mean())**2))
    print("R Squared :",1-(sum((pred-true)**2)/sum((true-true.mean())**2)))
    print("MSE :",((pred-true)**2).mean())
    print('MAE :',(abs(pred-true)).mean())
    print('Accuracy with 10% :', ((pred<=true*1.1) & (true*0.9<=pred)).mean())
    

def eval_report(y_train, pred_train,y_test, pred_test):
    reg_report(y_train, pred_train, name='Train')
    reg_report(y_test, pred_test, name='Test')
    
def res( name = 'Test'):  
    trues = []
    preds = []
    model.eval()
    with torch.no_grad():
        for row in range(len(torch.from_numpy(X_test).float())):
            features = torch.from_numpy(X_test).float()[row]
            price = torch.FloatTensor(y_test_NS.values).float()[row]

            outputs = model(features)
            #print(outputs)
            trues.append(price.tolist())
            preds.append(outputs.tolist())

    reg_report(true =  np.array(trues), pred = np.array(preds), name=name)

res( name = 'Test')
#res(test_dataloader, name = 'Train')

[19.0, 27.0, 16.0, 7.0, 2.0, 1.0, 37.0, 28.0, 32.0, 27.0, 33.0, 25.0, 8.0, 16.0, 8.0, 28.0, 8.0, 34.0, 23.0, 26.0, 8.0, 2.0, 15.0, 12.0, 35.0, 20.0, 21.0, 24.0, 17.0, 2.0, 38.0, 38.0, 0.0, 29.0, 25.0, 2.0, 2.0, 27.0, 18.0, 27.0, 35.0, 12.0, 16.0, 28.0, 16.0, 2.0, 24.0, 15.0, 3.0, 23.0, 30.0, 29.0, 23.0, 17.0, 16.0, 24.0, 33.0, 36.0, 4.0, 38.0, 12.0, 36.0, 24.0, 9.0, 25.0, 11.0, 38.0, 34.0, 5.0, 25.0, 15.0, 34.0, 32.0, 13.0, 35.0, 5.0, 11.0, 2.0, 2.0, 15.0, 37.0, 6.0, 24.0, 16.0, 2.0, 26.0, 7.0, 0.0, 27.0, 23.0, 33.0, 24.0, 11.0, 24.0, 7.0, 11.0, 3.0, 21.0, 26.0, 35.0, 38.0, 11.0, 19.0, 33.0, 31.0, 29.0, 29.0, 34.0, 11.0, 30.0, 5.0, 9.0, 8.0, 20.0, 21.0, 8.0, 36.0, 10.0, 1.0, 32.0, 20.0, 4.0, 36.0, 35.0, 0.0, 29.0, 21.0, 38.0, 22.0, 13.0, 20.0, 2.0, 12.0, 18.0, 0.0, 20.0, 35.0, 16.0, 10.0, 31.0, 9.0, 18.0, 34.0, 20.0, 13.0, 38.0, 3.0, 33.0, 21.0, 30.0, 23.0, 22.0, 9.0, 0.0, 34.0, 29.0, 6.0, 16.0, 26.0, 36.0, 26.0, 19.0, 9.0, 23.0, 5.0, 17.0, 32.0, 3.0, 16.0, 15.0, 16.0, 22.0, 17.0, 37.0

ValueError: operands could not be broadcast together with shapes (3900,38) (3900,) 